In [1]:
!python -m spacy download en_core_web_md -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 14.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
!pip install spacy==3.7.4 gensim==4.3.2 pandas==2.2.1 scikit-learn==1.4.1.post1 pyldavis==3.4.1 matplotlib==3.8.3 seaborn==0.13.2 jupyter==1.0.0 numpy==1.26.3 "scipy<1.13.0" "bertopic>=0.16.4,<0.17.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0

# Import

In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"  # see the value of multiple statements at once.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
import spacy
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()  # Visualise inside a notebook
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

assets_dir = 'assets'
workers = 6


# Collect Data

In [2]:
repos = pd.read_json(f'{assets_dir}/output.json')
repos.info()
repos.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15862 entries, 0 to 15861
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           15862 non-null  object
 1   owner          15862 non-null  object
 2   description    15862 non-null  object
 3   topics         15862 non-null  object
 4   url            15862 non-null  object
 5   readme_base64  15862 non-null  object
dtypes: object(6)
memory usage: 743.7+ KB


,name,owner,description,topics,url,readme_base64
0,hub,mislav,A command-line tool that makes git easier to u...,"[go, homebrew, git, github-api, pull-request]",https://github.com/mislav/hub,aHViIGlzIGEgY29tbWFuZCBsaW5lIHRvb2wgdGhhdCB3cm...
1,lantern,getlantern,Lantern官方版本下载 蓝灯 翻墙 代理 科学上网 外网 加速器 梯子 路由 - Быс...,"[lantern, vpn, censorship, circumvention, gfw,...",https://github.com/getlantern/lantern,IyBMYW50ZXJuClshW2VuXShyZXNvdXJjZXMvRW5nbGlzaC...
2,direnv,direnv,unclutter your .profile,"[direnv, environment, shell-extension, bash, z...",https://github.com/direnv/direnv,ZGlyZW52IC0tIHVuY2x1dHRlciB5b3VyIC5wcm9maWxlCj...
3,go-cache,patrickmn,An in-memory key:value store/cache (similar to...,"[go, cache, library]",https://github.com/patrickmn/go-cache,IyBnby1jYWNoZQoKZ28tY2FjaGUgaXMgYW4gaW4tbWVtb3...
4,dns,miekg,DNS library in Go,"[dnssec, go, dns-library, dns]",https://github.com/miekg/dns,WyFbQnVpbGQgU3RhdHVzXShodHRwczovL3RyYXZpcy1jaS...


# Preprocess data

In [3]:
from base64 import standard_b64decode
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

repos['readme_base64'] = repos['readme_base64'].apply(lambda x: standard_b64decode(x).decode('utf-8'))
repos['topics'] = repos['topics'].apply(lambda x: ' '.join(x))
repos['merged'] = repos['description'] + ' ' + repos['topics'] + ' ' + repos['readme_base64']

nlp = spacy.load('en_core_web_md')
removal = ['ADV', 'PRON', 'CCONJ', 'PUNCT', 'PART', 'DET', 'ADP', 'SPACE', 'NUM', 'SYM']

processed_texts = []
for merged_text in nlp.pipe(repos['merged'], batch_size=50):
    tokens = [token.lemma_.lower() for token in merged_text if token.pos_ not in removal and not token.is_stop and token.is_alpha]
    processed_texts.append(' '.join(tokens))


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
vectorizer = CountVectorizer(max_df=0.5, min_df=5, max_features=1000)
topic_model = BERTopic(vectorizer_model=vectorizer)
# topic_model = BERTopic(nr_topics=5)

topics, probs = topic_model.fit_transform(processed_texts)

repos['bertopic_topics'] = topics
repos['topic_probabilities'] = probs


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [10]:
topic_model.visualize_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

